In [1]:
import os
import sys
from pathlib import Path
import gc
import ee
import geemap
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import rasterio
from rasterio.mask import mask
from rasterio.enums import Resampling
from dataclasses import dataclass, field
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Patch
from matplotlib_scalebar.scalebar import ScaleBar
import warnings
import unicodedata
import re

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
warnings.filterwarnings("ignore")

@dataclass
class JRCConfig:
    # Caminhos relativos
    BASE_DIR: Path = Path("..")
    PATH_MUNICIPIOS: Path = BASE_DIR / "inputs" / "BR_Municipios_2024" / "BR_Municipios_2024.shp"
    PATH_ESTADOS: Path = BASE_DIR / "inputs" / "BR_UF_2024" / "BR_UF_2024.shp"
    PATH_OUTPUT: Path = BASE_DIR / "output"
    
    NOME_MUNICIPIO: str = 'Sobradinho'
    SIGLA_UF: str = 'BA'
    
    ASSET_ID: str = "JRC/GSW1_4/GlobalSurfaceWater"
    SCALE_METERS: int = 60
    
    CRS_VISUAL: str = 'EPSG:3857' 
    CRS_LATLON: str = 'EPSG:4674'
    
    TRANSITION_MAP: dict = field(default_factory=lambda: {
        0: {'label': 'Sem Água', 'color': '#ffffff'},
        1: {'label': 'Água Permanente', 'color': '#0000ff'},
        2: {'label': 'Nova Permanente', 'color': '#22b14c'},
        3: {'label': 'Perda Permanente', 'color': '#d1102d'},
        4: {'label': 'Sazonal', 'color': '#99d9ea'},
        5: {'label': 'Nova Sazonal', 'color': '#b5e61d'},
        6: {'label': 'Perda Sazonal', 'color': '#e6a1aa'},
        7: {'label': 'Sazonal -> Permanente', 'color': '#ff7f27'},
        8: {'label': 'Permanente -> Sazonal', 'color': '#ffc90e'},
        9: {'label': 'Efêmera Permanente', 'color': '#7f7f7f'},
        10: {'label': 'Efêmera Sazonal', 'color': '#c3c3c3'}
    })

class GeoUtils:
    @staticmethod
    def normalize_string(text: str) -> str:
        if not isinstance(text, str): return ""
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower().strip()

    @staticmethod
    def sanitize_filename(nome: str) -> str:
        return re.sub(r'[^\w\s-]', '', GeoUtils.normalize_string(nome)).strip().replace(' ', '_')

class JRCService:
    def __init__(self, config: JRCConfig):
        self.cfg = config
        self._authenticate()

    def _authenticate(self):
        try:
            ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
        except:
            ee.Authenticate()
            ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

    def get_water_data(self, region_ee):
        jrc = ee.Image(self.cfg.ASSET_ID)
        occurrence = jrc.select('occurrence')
        transition = jrc.select('transition')
        
        pixel_area = ee.Image.pixelArea().divide(1e6)
        stats = pixel_area.addBands(transition).reduceRegion(
            reducer=ee.Reducer.sum().group(groupField=1, groupName='class_code'),
            geometry=region_ee.geometry(),
            scale=self.cfg.SCALE_METERS, 
            maxPixels=1e13,
            bestEffort=True
        )
        
        groups = stats.get('groups').getInfo()
        df_data = []
        if groups:
            for item in groups:
                code = int(item['class_code'])
                area = item['sum']
                if code in self.cfg.TRANSITION_MAP and code != 0:
                    meta = self.cfg.TRANSITION_MAP[code]
                    df_data.append({
                        'Class_ID': code,
                        'Classe': meta['label'],
                        'Color': meta['color'],
                        'Area_km2': area
                    })
        
        export_img = occurrence.unmask(0).rename('occurrence') \
            .addBands(transition.unmask(0).rename('transition')) \
            .clip(region_ee.geometry())
            
        return export_img, pd.DataFrame(df_data).sort_values('Area_km2', ascending=False)

    def download_raster(self, image: ee.Image, region_ee, output_path: Path):
        if output_path.exists():
            try: os.remove(output_path)
            except: pass

        geemap.download_ee_image(
            image,
            filename=str(output_path),
            scale=self.cfg.SCALE_METERS,
            region=region_ee.geometry(),
            crs=self.cfg.CRS_VISUAL,
            overwrite=True,
            num_threads=4 
        )

class JRCRenderer:
    def __init__(self, config: JRCConfig):
        self.cfg = config

    def _styled_box(self, ax, title):
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
            spine.set_edgecolor('#333333')
            spine.set_linewidth(1)
        ax.set_title(title, fontsize=9, weight='bold', pad=6, backgroundcolor='#e0e0e0', color='black')

    def _setup_grid_lines(self, ax, raster_bounds):
        minx, miny, maxx, maxy = raster_bounds
        x_target = np.linspace(minx, maxx, 4)
        y_target = np.linspace(miny, maxy, 4)
        
        pts_metric = gpd.points_from_xy(x_target, y_target, crs=self.cfg.CRS_VISUAL)
        pts_latlon = pts_metric.to_crs(self.cfg.CRS_LATLON)
        
        ax.set_xticks(x_target)
        ax.set_yticks(y_target)
        
        x_lbl = [f"{p.x:.2f}°" for p in pts_latlon]
        y_lbl = [f"{p.y:.2f}°" for p in pts_latlon]
        
        ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: x_lbl[p] if p < len(x_lbl) else ""))
        ax.yaxis.set_major_formatter(FuncFormatter(lambda y, p: y_lbl[p] if p < len(y_lbl) else ""))
        ax.tick_params(axis='both', which='major', labelsize=7, direction='in', pad=4)
        ax.grid(True, linestyle='--', alpha=0.3, color='#444444', linewidth=0.5, zorder=3)

    def _draw_minimap(self, ax, mun_gdf):
        self._styled_box(ax, f"LOCALIZAÇÃO - {self.cfg.SIGLA_UF}")
        try:
            if self.cfg.PATH_ESTADOS.exists():
                states = gpd.read_file(self.cfg.PATH_ESTADOS)
                state = states[states['SIGLA_UF'] == self.cfg.SIGLA_UF.upper()]
                if not state.empty:
                    state.to_crs(self.cfg.CRS_VISUAL).plot(ax=ax, facecolor='#f4f4f4', edgecolor='#555555', linewidth=0.5)
                    mun_gdf.to_crs(self.cfg.CRS_VISUAL).plot(ax=ax, facecolor='#d62728', edgecolor='none')
            else:
                ax.text(0.5, 0.5, "Shape UF N/A", ha='center', fontsize=7)
        except: pass

    def _draw_metadata(self, ax):
        self._styled_box(ax, "")
        ax.text(0.5, 0.95, "METADADOS", ha='center', va='top', fontsize=9, weight='bold', transform=ax.transAxes)
        txt = (
            f"Município: {self.cfg.NOME_MUNICIPIO}\n"
            f"Dataset: JRC Global Surface Water v1.4\n"
            f"Período: 1984 - 2021\n"
            f"Resolução: {self.cfg.SCALE_METERS}m\n"
            f"Processamento: GEE & Python"
        )
        ax.text(0.5, 0.75, txt, ha='center', va='top', fontsize=8, linespacing=1.6, transform=ax.transAxes)

    def generate_map(self, raster_path, mun_gdf, band_idx, title, cmap_type='continuous'):
        fig = plt.figure(figsize=(11.69, 8.27))
        gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1], wspace=0.1, figure=fig)
        
        ax_map = fig.add_subplot(gs[0])
        
        with rasterio.open(raster_path) as src:
            overscale = max(1, src.width // 2000)
            data = src.read(band_idx, out_shape=(src.height // overscale, src.width // overscale), resampling=Resampling.nearest)
            data = np.where(data == 0, np.nan, data)
            extent = (src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top)

            if cmap_type == 'continuous':
                im = ax_map.imshow(data, cmap='Blues', vmin=0, vmax=100, extent=extent, zorder=1)
                unit_label = "Frequência (%)"
            else:
                max_val = 10
                colors = [self.cfg.TRANSITION_MAP[i]['color'] for i in range(max_val + 1)]
                cmap = ListedColormap(colors)
                norm = BoundaryNorm(np.arange(-0.5, max_val + 1.5, 1), cmap.N)
                im = ax_map.imshow(data, cmap=cmap, norm=norm, extent=extent, zorder=1, interpolation='nearest')
                unit_label = "Classe"

            self._setup_grid_lines(ax_map, src.bounds)

        mun_proj = mun_gdf.to_crs(self.cfg.CRS_VISUAL)
        mun_proj.plot(ax=ax_map, facecolor='none', edgecolor='black', linewidth=1.5, linestyle='--', zorder=5)

        ax_map.add_artist(ScaleBar(1, units='m', location='lower left', box_alpha=0.7, font_properties={'family': 'serif', 'size': 8}))
        ax_map.annotate('N', xy=(0.95, 0.95), xytext=(0.95, 0.88), 
                        arrowprops=dict(facecolor='black', width=4, headwidth=10),
                        ha='center', va='center', fontsize=12, weight='bold', xycoords='axes fraction', zorder=10)
        ax_map.set_title(f"{title.upper()}\n{self.cfg.NOME_MUNICIPIO} - {self.cfg.SIGLA_UF}", fontsize=14, weight='bold', pad=15)

        gs_side = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs[1], height_ratios=[0.3, 0.4, 0.3], hspace=0.3)
        self._draw_minimap(fig.add_subplot(gs_side[0]), mun_gdf)
        
        ax_leg = fig.add_subplot(gs_side[1])
        self._styled_box(ax_leg, "LEGENDA")
        
        if cmap_type == 'continuous':
            cbar = plt.colorbar(im, cax=ax_leg, orientation='horizontal', fraction=0.5, pad=0.1)
            cbar.set_label(unit_label, fontsize=8, weight='bold')
        else:
            ax_leg.axis('off')
            legend_patches = []
            for code, meta in self.cfg.TRANSITION_MAP.items():
                if code == 0: continue
                legend_patches.append(Patch(facecolor=meta['color'], edgecolor='#333333', label=meta['label']))
            ax_leg.legend(handles=legend_patches, loc='center', title="", frameon=False, fontsize=8)

        self._draw_metadata(fig.add_subplot(gs_side[2]))
        return fig

    def generate_stats_page(self, df):
        fig = plt.figure(figsize=(11.69, 8.27))
        fig.suptitle(f"ESTATÍSTICAS JRC - {self.cfg.NOME_MUNICIPIO}", fontsize=16, weight='bold', y=0.95)
        ax = fig.add_subplot(111)
        
        if not df.empty:
            colors = df['Color'].tolist()
            bars = ax.barh(df['Classe'], df['Area_km2'], color=colors, edgecolor='black', alpha=0.9)
            ax.invert_yaxis()
            ax.set_xlabel("Área Total (km²)", fontsize=10, weight='bold')
            ax.grid(True, axis='x', linestyle='--', alpha=0.3)
            
            max_val = df['Area_km2'].max()
            for bar in bars:
                width = bar.get_width()
                label_x_pos = width + (max_val * 0.01)
                ax.text(label_x_pos, bar.get_y() + bar.get_height()/2, f'{width:,.2f} km²', va='center', fontsize=9, fontweight='bold')
        else:
            ax.text(0.5, 0.5, "Nenhuma água detectada.", ha='center', fontsize=12)
            ax.axis('off')
        return fig

class JRCPipeline:
    def __init__(self, config: JRCConfig):
        self.cfg = config
        self.service = JRCService(config)
        self.renderer = JRCRenderer(config)

    def run(self):
        temp_tif = None
        try:
            print(f"--- INICIANDO JRC: {self.cfg.NOME_MUNICIPIO} ---")
            
            full_gdf = gpd.read_file(self.cfg.PATH_MUNICIPIOS)
            full_gdf['norm_nm'] = full_gdf['NM_MUN'].apply(GeoUtils.normalize_string)
            aoi = full_gdf[(full_gdf['norm_nm'] == GeoUtils.normalize_string(self.cfg.NOME_MUNICIPIO)) & 
                           (full_gdf['SIGLA_UF'] == self.cfg.SIGLA_UF.upper())].dissolve()
            
            if aoi.empty: raise ValueError("Município não encontrado.")
            
            print("Processando Global Surface Water...")
            aoi_ee = geemap.geopandas_to_ee(aoi.to_crs("EPSG:4326"))
            img_jrc, df_stats = self.service.get_water_data(aoi_ee)
            
            safe_name = GeoUtils.sanitize_filename(self.cfg.NOME_MUNICIPIO)
            self.cfg.PATH_OUTPUT.mkdir(parents=True, exist_ok=True)
            temp_tif = self.cfg.PATH_OUTPUT / f"TEMP_JRC_{safe_name}.tif"
            
            print("Baixando Raster...")
            self.service.download_raster(img_jrc, aoi_ee, temp_tif)
            
            if not temp_tif.exists(): raise FileNotFoundError("Erro no download.")

            print("Gerando Mapas...")
            figs = []
            figs.append(self.renderer.generate_map(temp_tif, aoi, 1, "Frequência de Água (1984-2021)", 'continuous'))
            figs.append(self.renderer.generate_map(temp_tif, aoi, 2, "Dinâmica e Transição", 'categorical'))
            figs.append(self.renderer.generate_stats_page(df_stats))
            
            pdf_path = self.cfg.PATH_OUTPUT / f"RELATORIO_JRC_{safe_name}.pdf"
            with PdfPages(pdf_path) as pdf:
                for f in figs:
                    pdf.savefig(f, bbox_inches='tight', dpi=300)
                    plt.close(f)
            
            print(f"Relatório Salvo: {pdf_path}")

        except Exception as e:
            print(f"ERRO CRÍTICO: {e}")
            import traceback; traceback.print_exc()
        finally:
            if temp_tif and temp_tif.exists():
                try: 
                    gc.collect() 
                    temp_tif.unlink()
                except: pass

if __name__ == "__main__":
    cfg = JRCConfig(NOME_MUNICIPIO='Três Corações', SIGLA_UF='MG')
    JRCPipeline(cfg).run()

--- INICIANDO JRC: Três Corações ---
Processando Global Surface Water...
Baixando Raster...
ERRO CRÍTICO: The EPSG code is unknown. PROJ: proj_create_from_database: C:\Program Files\PostgreSQL\16\share\contrib\postgis-3.4\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.


Traceback (most recent call last):
  File "rasterio\\crs.pyx", line 592, in rasterio.crs.CRS.from_epsg
  File "rasterio\\_err.pyx", line 289, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: PROJ: proj_create_from_database: C:\Program Files\PostgreSQL\16\share\contrib\postgis-3.4\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\pedro\AppData\Local\Temp\ipykernel_22436\1782738789.py", line 291, in run
    self.service.download_raster(img_jrc, aoi_ee, temp_tif)
  File "C:\Users\pedro\AppData\Local\Temp\ipykernel_22436\1782738789.py", line 122, in download_raster
    geemap.download_ee_image(
  File "c:\Users\pedro\Downloads\python_gis\.venv\lib\site-packages\geemap\common.py", line 12537, in download_ee_image
    img.download(filename, overwrite=overwrite, num_threads=n